In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

# from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.1,
)

In [2]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

In [3]:
# LangChain Expression Language(LCEL)를 이용해서 생성된 체인에 메모리를 추가하는 것은 어렵지 않음
# 실제로 변경 작업을 할 때 권장되는 방법이고, 현재 Langchain Expression 언어에서 동작하는 방법임

# 나중에는 메모리를 추가하는게 LLM 체인을 사용하는 것만큼이나 더 쉬워질 것 -> 지금은 manual 작업을 통해 구성해본거

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,  # 문자열말고 message로 출력
)

# 중요한 점은
# 메모리가 변경되지 않는다면, prompt도 변하지 않음

In [5]:
# LLMChain이 아닌 그냥 chain을 만들어주고,
chain = prompt | llm

chain.invoke(
    {
        # .predict가 아니라 invoke로 넘겨줌
        "chat_history": memory.load_memory_variables({})["chat_history"],
        "question": "My name is Nico.",  # 여기에 human 메시지 넣어서 전달
    }
)

# 이렇게 했을때 문제점은 체인을 호출할떄마다 chat_history도 추가해줘야함

AIMessage(content='Nice to meet you, Nico! How can I assist you today?')

In [20]:
def load_memory(input):
    # RunnablePassthrough는 기본적으로 1개의 argument를 가짐 -> invoke에 전달하는 dictionary
    # 왜냐면 chain에 있는 모든 component는 input을 받고 output을 넘겨주기 때문에
    # print(input)
    return memory.load_memory_variables({})["chat_history"]

In [15]:
# 더 좋은 방법 -> RunnablePassthrough
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm
# chain을 실행하면, 가장 먼저 load_memory 함수를 호출함
# 그리고 기본적으로 prompt가 필요로 하는 chat_history key 내부에 넣어줄거
# chain.invoke할때 내부에 "chat_history":load_memory() 해주는거랑 같음

# 기본적으로 RunnablePassthrough는 prompt가 format되기 전에 우리가 함수를 실행시키는걸 허락해줌
# .assign을 통해서 우리가 원하는 어떤 값이든 변수에 할당할 수 있음

chain.invoke(
    {
        # 이 부분은 chain의 첫번째 아이템 input이 됨
        "question": "My name is Nico.",  # 여기에 human 메시지 넣어서 전달
    }
)

{'question': 'My name is Nico.'}


AIMessage(content='Nice to meet you, Nico! How can I assist you today?')

In [17]:
# chain이 구성되었으면, 이제 각 대화의 결과를 메모리에 저장할 수 있어야함
# memory.load_memory_variables({}) # {'chat_history': []} -> 지금은 저장되고 있지 않음
# 지금은 메모리를 수동으로 관리하기 때문

# 가장 좋은 방법은 체인을 호출하는 함수를 만드는 거


def invoke_chain(question):
    result = chain.invoke(
        {
            "question": question,
        }
    )
    memory.save_context(
        {"input": question},  # input은 사용자의 질문이 될거고
        {"output": result.content},  # output은 그에 대한 결과
    )
    print(result)

In [21]:
invoke_chain("My name is Nico")

{'question': 'My name is Nico'}
content='Nice to meet you, Nico! How can I assist you today?'


In [22]:
invoke_chain("What is my name?")

{'question': 'What is my name?'}
content='Your name is Nico.'
